In [12]:
    import pandas as pd
    
    from bokeh.io import output_notebook
    from bokeh.io import show
    from bokeh.models import ColumnDataSource
    from bokeh.plotting import gmap
    from bokeh.models import GMapOptions
    from bokeh.models import HoverTool
    from bokeh.io import export_png
    from bokeh.transform import linear_cmap
    from bokeh.palettes import Plasma256 as palette
    from bokeh.models import ColorBar
    
    bokeh_width, bokeh_height = 500,400

In [13]:
df = pd.read_csv('C:/Users/dasus/Documents/NCAS-M/NCAS/Data/depth_data/Mar-25-2022.csv')
df.head()

,Latitude,Longitude,Depth
0,18.099450,-67.190283,1.51
1,18.099433,-67.190267,1.51
2,18.099433,-67.190267,1.51
3,18.099433,-67.190267,1.51
4,18.099433,-67.190267,1.51


In [14]:
lat, lon= df.Latitude[0], df.Longitude[0]


In [15]:


import os 
api_key = os.environ['GOOGLE_API_KEY']



In [16]:

import numpy as np
df['radius'] = np.sqrt(df['Depth'])/0.8
df.head()

,Latitude,Longitude,Depth,radius
0,18.099450,-67.190283,1.51,1.536026
1,18.099433,-67.190267,1.51,1.536026
2,18.099433,-67.190267,1.51,1.536026
3,18.099433,-67.190267,1.51,1.536026
4,18.099433,-67.190267,1.51,1.536026


In [17]:


from bokeh.transform import linear_cmap
from bokeh.palettes import Plasma256 as palette
from bokeh.models import ColorBar

# we are adding the dataframe as a parameter, 
# since we are now going to plot
# a different dataframe
def plot(df, lat, lng, zoom=10, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    hover = HoverTool(
        tooltips = [
            ('Depth in Feet', '@Depth '),
            # the {0.} means that we don't want decimals
            # for 1 decimal, write {0.0}
        ]
    )
    p = gmap(api_key, gmap_options, title='Bathymetry Map Parguera', 
             width=bokeh_width, height=bokeh_height,
             tools=[hover, 'reset', 'wheel_zoom', 'pan'])
    source = ColumnDataSource(df)
    # defining a color mapper, that will map values of pricem2
    # between 2000 and 8000 on the color palette
    mapper = linear_cmap('Depth', palette, min(df.Depth), max(df.Depth))    
    # we use the mapper for the color of the circles
    center = p.circle('Longitude', 'Latitude', radius='radius', alpha=0.4, 
                      color=mapper, source=source)
    # and we add a color scale to see which values the colors 
    # correspond to 
    color_bar = ColorBar(color_mapper=mapper['transform'], 
                         location=(0,0))
    p.add_layout(color_bar, 'right')
    
    p.background_fill_color = None
    p.border_fill_color = None
    
    

    export_png(p, filename="plot.png")   
    return p

p = plot(df, lat, lon, map_type='satellite', zoom=18)

